In [1]:
import numpy as np
import pandas as pd

In [2]:
from gensim.models import Word2Vec

C:\Users\k\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from keras.models import Sequential, load_model
from keras.layers import Conv1D, Dense, Dropout, Activation, Embedding, GlobalMaxPool1D
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Users\k\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
max_len = 500
embedding_size = 100
dropout=0.5
epochs = 10
validation_split = 0.1
batch_size = 128
filters= 200 
kernel_size= 3
num_words = 100000

In [5]:
tokenizer = Tokenizer(num_words=num_words)

In [6]:
train = pd.read_pickle('train.pkl')

In [7]:
tokenizer.fit_on_texts(train.Content)

In [8]:
vocab_size = len(tokenizer.word_index)

In [9]:
word_vec = Word2Vec.load('./word2vec')

In [10]:
vocab_size

878856

In [11]:
train_seq = tokenizer.texts_to_sequences(train.Content)

In [12]:
X_train = pad_sequences(train_seq, maxlen=max_len, truncating='post', padding='post')

In [13]:
y_train = pd.get_dummies(train.Lable).values

In [14]:
n_class = y_train.shape[1]

In [15]:
embedding_matrix = np.zeros((vocab_size, embedding_size), dtype=np.float32)

In [16]:
unknown_count = 0
unknown_freq = {}
for word, index in tokenizer.word_index.items():
    if index >= vocab_size: 
            continue
    try:
        embedding_matrix[index, :] = word_vec.wv[word]
    except KeyError:
        unknown_freq[word] = tokenizer.word_counts[word]
        unknown_count += 1

In [17]:
def computation_graph():
    model  = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], name='Embedding_Layer'))
    model.add(Conv1D(filters= filters, 
                     kernel_size= kernel_size,
                     activation = 'relu',
                     name= '_'.join(['Convolution_1D', str(filters), str(kernel_size)])))
    model.add(GlobalMaxPool1D(name='Global_Max_Pooling'))
    model.add(Dense(units=filters, name='Dense_'+str(filters)))
    model.add(Dropout(rate=dropout, name = 'Dropout_' + str(dropout)))
    model.add(Activation('relu', name='Activation_'+str('relu')))
    model.add(Dense(units=n_class, activation='softmax', name='Dense_'+str(n_class)+'_Sigmoid'))
    return model

In [18]:
model = computation_graph()

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, None, 100)         87885600  
_________________________________________________________________
Convolution_1D_200_3 (Conv1D (None, None, 200)         60200     
_________________________________________________________________
Global_Max_Pooling (GlobalMa (None, 200)               0         
_________________________________________________________________
Dense_200 (Dense)            (None, 200)               40200     
_________________________________________________________________
Dropout_0.5 (Dropout)        (None, 200)               0         
_________________________________________________________________
Activation_relu (Activation) (None, 200)               0         
_________________________________________________________________
Dense_14_Sigmoid (Dense)     (None, 14)                2814      
Total para

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
history = model.fit(x=X_train, y=y_train,
                          validation_split = validation_split,
                          epochs=epochs,
                          batch_size=batch_size, 
                          shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 34s 760us/step - loss: 0.9637 - acc: 0.7016 - val_loss: 0.5476 - val_acc: 0.8272
Epoch 2/10
45000/45000 [==============================] - 31s 696us/step - loss: 0.7193 - acc: 0.7729 - val_loss: 2.1145 - val_acc: 0.3092
Epoch 3/10
45000/45000 [==============================] - 34s 759us/step - loss: 1.9882 - acc: 0.3032 - val_loss: 1.9579 - val_acc: 0.3092
Epoch 4/10
45000/45000 [==============================] - 32s 710us/step - loss: 1.9776 - acc: 0.3038 - val_loss: 1.9576 - val_acc: 0.3092
Epoch 5/10
45000/45000 [==============================] - 32s 711us/step - loss: 1.9749 - acc: 0.3038 - val_loss: 1.9572 - val_acc: 0.3092
Epoch 6/10
45000/45000 [==============================] - 33s 743us/step - loss: 1.9737 - acc: 0.3038 - val_loss: 1.9572 - val_acc: 0.3092
Epoch 7/10
45000/45000 [==============================] - 33s 726us/step - loss: 1.9723 - acc: 0.3038 - val_loss: 1.9